# 整合 Q1～Q4 的維修預測分析

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


## Q1: 資料前處理

In [ ]:
# 讀取資料
df = pd.read_csv("maintenance_prediction.csv")

# 檢查缺值與類型
print(df.info())

# 將特徵與標籤分開
X = df.drop(columns=["failure"])
y = df["failure"]


## Q2: 找出重要變數

In [ ]:
# 用 Random Forest 訓練來抓特徵重要性
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# 顯示變數重要性
importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

# 畫圖
plt.figure(figsize=(10,6))
sns.barplot(x="importance", y="feature", data=importance_df)
plt.title("Feature Importance")
plt.tight_layout()
plt.show()


## Q3: 建立與訓練模型

In [ ]:
# 分訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 用最佳模型訓練
best_model = RandomForestClassifier(n_estimators=100, random_state=42)
best_model.fit(X_train, y_train)

# 測試集預測與評估
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_proba))


## Q4: 預測新資料

In [ ]:
# 建立新測試資料（Test1 與 Test2）
test_data = pd.DataFrame({
    "metric1": [127175526, 4527376],
    "metric2": [4109.434, 0],
    "metric3": [125581.3, 0],
    "metric4": [29999.98, 0],
    "metric5": [201.79, 0],
    "metric6": [16491.86, 0],
    "metric7": [0, 0],
    "metric8": [6.0, 0],
    "metric9": [100.0, 0]
}, index=["Test1", "Test2"])

# 預測
pred_class = best_model.predict(test_data)
pred_proba = best_model.predict_proba(test_data)[:, 1]

# 結果整理
results = pd.DataFrame({
    "Predicted Failure": pred_class,
    "Failure Probability": pred_proba
}, index=["Test1", "Test2"])

print("預測結果：")
print(results)
